In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import os, sys

In [3]:
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))

In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [5]:
%pwd

u'/home/ubuntu/nbs/lesson5'

In [6]:
path = 'data/imdb/models/'
%mkdir -p $path

## Preparing data

In [7]:
from keras.datasets import imdb
word2idx = imdb.get_word_index()
idx2word = {v: k for k, v in word2idx.items()}

In [21]:
word2idx

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [8]:
# sort by index or by freq?
idx_array = sorted(word2idx, key=word2idx.get) # what about add reverse=True
idx_array[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [9]:
(x_train, y_train), (x_test, y_test) = imdb.load_data()

In [10]:
len(x_train)

25000

In [11]:
len(x_test)

25000

In [12]:
', '.join(map(str, x_train[0]))

'1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32'

In [20]:
idx2word[3000]

'stops'

In [22]:
' '.join(idx2word[i] for i in x_train[0])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

In [23]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [32]:
vocab_size=8000
trn = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in review]) for review in x_train]
test = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in review]) for review in x_test]

In [33]:
trn_len = np.array(map(len, trn))
(trn_len.max(), trn_len.min(), trn_len.mean())

(2494, 11, 238.71364)

In [34]:
seq_len = 500
trn = sequence.pad_sequences(trn, maxlen=seq_len) # pad with 0 by default
test = sequence.pad_sequences(test, maxlen=seq_len)

In [35]:
trn.shape

(25000, 500)

## Single hidden layer

In [40]:
def single_hidden_layer():
    inp = Input(shape=(seq_len,), dtype='int64')
    x = Embedding(vocab_size, 50, input_length=seq_len)(inp)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activatioin='sigmoid')(x) # sigmoid because it's binary classification
    
    model = Model(inp, x)
    model.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [41]:
model = single_hidden_layer()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 500, 50)       400000      input_2[0][0]                    
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25000)         0           embedding_3[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 100)           2500100     flatten_2[0][0]                  
___________________________________________________________________________________________

In [42]:
model.fit(trn, y_train, batch_size=64, nb_epoch=3, validation_data=(test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 15s - loss: 4.2939 - acc: 0.3294 - val_loss: 0.5268 - val_acc: 0.7242
Epoch 2/3
25000/25000 [==============================] - 16s - loss: 0.5085 - acc: 0.7631 - val_loss: 0.5271 - val_acc: 0.7891
Epoch 3/3
25000/25000 [==============================] - 16s - loss: 0.3072 - acc: 0.7425 - val_loss: 0.5670 - val_acc: 0.7283


## Single conv layer

In [49]:
def single_conv_layer():
    inp = Input(shape=(seq_len,), dtype='int64')
    x = Embedding(vocab_size, 50, input_length=seq_len)(inp)
    x = Convolution1D(64, 5, border_mode='same', activation='relu')
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1)(x)
    
    model = Model(inp, x)
    model.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = single_conv_layer()
model.summary()

In [51]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [52]:
model.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_8 (Embedding)          (None, 500, 32)       256000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 500, 32)       0           embedding_8[0][0]                
____________________________________________________________________________________________________
convolution1d_5 (Convolution1D)  (None, 500, 64)       10304       dropout_4[0][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 500, 64)       0           convolution1d_5[0][0]            
___________________________________________________________________________________________

In [53]:
model.fit(trn, y_train, batch_size=64, nb_epoch=3, validation_data=(test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 31s - loss: 0.4969 - acc: 0.7233 - val_loss: 0.3204 - val_acc: 0.8648
Epoch 2/3
25000/25000 [==============================] - 31s - loss: 0.2750 - acc: 0.8950 - val_loss: 0.2713 - val_acc: 0.8900
Epoch 3/3
25000/25000 [==============================] - 31s - loss: 0.2328 - acc: 0.9115 - val_loss: 0.2602 - val_acc: 0.8914


In [54]:
model.save_weights(path+'conv.h5')
model.load_weights(path+'conv.h5')

In [62]:
model.predict(test)

array([[ 0.9474],
       [ 0.992 ],
       [ 0.9814],
       ..., 
       [ 0.6305],
       [ 0.0013],
       [ 0.7678]], dtype=float32)

In [63]:
' '.join(idx2word[i] for i in test[1])

"for ending total about those in acting this of intentions br is power linda constant movie is inevitable sees pretty in at seconds this ralph tend one in another american in also in is homicide this actually lethal know ealing honestly level me will smooth film of ealing street in swimming were duty those first others tend of tend br ealing ealing chaplin fail be hardly ealing ealing ealing like he give are of actress or revelation 70's paper br happens styles ealing audience ken into people jump i i viewer who ealing ealing was did inevitable br ealing of guy who attractive ealing it him played time future romantic fight overall lewis brings br because fact you film very kid's or of today paul know bad can wearing ealing ealing covered or of false of resist many chick ealing mum ethan ealing or of is actions are polished hot original close illegal covered or of relate separate in of legs short or inevitable br reid ealing no that hopefully ealing of every scenes inevitable br ealing 

In [74]:
' '.join(idx2word[i] for i in test[0])

KeyError: 0

## Pre-trained embedding

In [92]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [93]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [94]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

Untaring file...


In [95]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [96]:
emb = create_emb()

KeyError: 'gypo'

In [90]:
def pre_trained_emb_model():
    inp = Input(shape=(seq_len,), dtype='int64')
    x = Embedding(vocab_size, 50, input_length=seq_len, weights=[emb], trainable=False)(inp)
    x = Convolution1D(64, 3, activation='relu')
    x = Dropout(0.7)(x)
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(1)(x)
    
    model = Model(inp, x)
    model.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [91]:
model = pre_trained_emb_model()
model.summary()

NameError: global name 'emb' is not defined

In [ ]:
model.fit(trn, y_train, batch_size=64, nb_epoch=3, validation_data=(test, y_test))

In [ ]:
model.layers[0].trainable = True

In [ ]:
model.fit(trn, y_train, batch_size=64, nb_epoch=3, validation_data=(test, y_test))

In [ ]:
model.save_weights(path+'glove.h5')